In [29]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from pinecone import Pinecone as PC
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import os
from dotenv import load_dotenv

In [24]:
load_dotenv()

True

In [3]:
# Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data, glob='*.pdf', loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents


In [4]:

loaded_texts = load_pdf('./data/')

In [5]:
loaded_texts[5].page_content

'Encyclopedia of Business and Finance, Second Edition \nBurton S. Kaliski, Editor in Chief \n© 2007 Thomson Gale, a part of The Thomson\nCorporation.\nThomson, Star Logo and Macmillan Reference\nUSA are trademarks and Gale is a registered\ntrademark used herein under license.  \nFor more information, contactMacmillan Reference USAAn imprint of Thomson Gale27500 Drake Rd.Farmington, Hills, MI 48331-3535Or you can visit our Internet site athttp://www.gale.com\nALL RIGHTS RESERVED\nNo part of this work covered by the copyrighthereon may be reproduced or used in anyform or by any means—graphic, electronic, ormechanical, including photocopying, record-ing, taping, Web distribution, or informationstorage retrieval systems—without the writtenpermission of the publisher.  For permission to use material from this\nproduct, submit your request via Web athttp://www.gale-edit.com/permissions, or youmay download our Permissions Request formand submit your request by fax or mail to:\nPermissions\nTh

In [9]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks = text_split(loaded_texts)
print("length of my text chunks: ", len(text_chunks))

length of my text chunks:  10646


In [13]:
# download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = download_hugging_face_embeddings()

In [15]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [19]:
query_result = embeddings.embed_query("hello world")
len(query_result)

384

In [34]:
pc = PC(api_key=os.environ['PINECONE_API_KEY'])
index_name = "medical-chatbot"
index = pc.Index(index_name)

doc_search = Pinecone.from_documents(text_chunks, embeddings, index_name=index_name)